In [46]:
import re
import json
import subprocess

from openai import OpenAI
import sys
sys.path.append('../')
from prompts.design_prompt import design_prompt
from prompts.parsing_prompt import parsing_prompt
from prompts.scripting_prompt import scripting_prompt

In [4]:
with open('../config/config.json') as f:
     config = json.load(f)

client = OpenAI(api_key=config['OPENAI_API_KEY'])

In [16]:
design_description = f""" 
Suppose there is a one-stage system with two helical gears with a 30 degree pitch angle.
Their gear axes are parallel and the distance between two axes is 50mm. 
The height of both gears are 20 and 15 degree respectively.
Currently, there is a requirements for the reduction system that the desired gear ratio is 1.5:1, 
Set the driving gear to the center of the plane, which is the coordinate (0, 0)
Based on the given information, computer module number, teeth number of both gears and the coordniate of the driven gear centers.
"""

In [17]:
design_messages=[
        {"role": "system", "content": design_prompt},
        {"role": "user", "content": design_description},
    ]

In [18]:
# stop_tokens = ['']
# collected_tokens = []

# res = client.chat.completions.create(
#     model=config['GPT_MODEL'],
#     messages=design_messages,
#     stream=True,
# )

# for chunk in res:
#     temp_tokens = chunk.choices[0].delta.content
#     if temp_tokens not in stop_tokens:
#         collected_tokens.append(temp_tokens)
#     else:
#         chunk_res = "".join(collected_tokens).strip()
#         print(chunk_res)
#         collected_tokens = []

res = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=design_messages,
    # temperature=0
)

In [19]:
print(res.choices[0].message.content)

To design this gear system, we need to establish the basic geometric and kinematic relationships between the two gears. Here's how we can compute the parameters step by step.

Given the following information:
- Helical gears with a 30-degree pitch angle (α)
- Center distance (A) between gear axes: 50mm
- Gear ratio (i): 1.5:1
- The height of gear teeth are irrelevant in gear ratio calculations
- Driving gear centered at coordinate (0, 0)

### Step 1: Determine the Gear Ratio
The gear ratio (i) is given by the ratio of the number of teeth on the driven gear (N2) to the number of teeth on the driving gear (N1):

\[ i = \frac{N2}{N1} \]

With the desired gear ratio being 1.5:1, we get:

\[ i = 1.5 = \frac{N2}{N1} \]

### Step 2: Calculate the Number of Teeth
For standard gear systems, the number of teeth should ideally be a reasonable whole number to make manufacturing simpler. The precise number of teeth depends on other factors, including the module and pitch diameter.

### Step 3: Calc

In [20]:
with open("../common/helix_gear/gear_design.md", "w") as f:
    f.write(res.choices[0].message.content)

In [26]:
parsing_description = \
f"gear 1 (driving gear): helical gear, module 2mm, 20 teeth, pitch diameter 40mm, coordinate (0, 0)" + \
f"pitch angle 30 degrees, pressure angle 20, height 20mm" + \
f"gear 2 (driven gear): helical gear, module 2mm, 30 teeth, pitch diameter 60mm, coordinate (50, 0)" + \
f"pitch angle 30 degrees, pressure angle 20, height 15mm"

In [27]:
parsing_messages = [
    {"role": "system", "content": parsing_prompt},
    {"role": "user", "content": parsing_description},
]

In [28]:
parsing_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    response_format={ "type": "json_object" },
    messages=parsing_messages,
)

In [29]:
gears_parameters = parsing_response.choices[0].message.content.strip()
print(gears_parameters)

{
  "gear 1": {
    "gear_type": "helix",
    "coordinate": {
      "x": 0,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 20,
    "height": 20,
    "pitch_d": 40,
    "pitch_angle": 30,
    "pressure_angle": 20
  },
  "gear 2": {
    "gear_type": "helix",
    "coordinate": {
      "x": 50,
      "y": 0,
      "z": 0,
      "alpha": 0,
      "beta": 0,
      "gamma": 0
    },
    "unit": "mm",
    "module": 2,
    "teeth": 30,
    "height": 15,
    "pitch_d": 60,
    "pitch_angle": 30,
    "pressure_angle": 20
  }
}


In [30]:
with open("../common/helix_gear/gears_parameters.json", "w") as f:
    f.write(gears_parameters)

In [42]:
with open("../common/helix_gear/gears_parameters.json", "r") as f:
    parameters = json.load(f)
parameters = str(parameters)

scripting_description = f"Write a script according to provided procedure with parameters of gears" + parameters

In [43]:
script_messages = [
    {"role": "system", "content": scripting_prompt},
    {"role": "user", "content": scripting_description}
]

In [44]:
scripting_response = client.chat.completions.create(
    model=config['GPT_MODEL'],
    messages=script_messages,
    temperature=0
)

In [47]:
script = scripting_response.choices[0].message.content.strip()
# print(script)

# Define the regular expression pattern to match the openscad code block
pattern = re.compile(r'```openscad(.*?)```', re.DOTALL)

# Use findall to get all matches
matches = re.findall(pattern, script)

# Print the extracted code blocks
for match in matches:
    print(match.strip())

// Gear 1
translate([0, 0, 0]) rotate([0, 0, 360 / (20 * 2)]) gear_helix(m = 2, z = 20, x = 0, h = 20, w = 20, w_helix = 30);

// Gear 2
translate([50, 0, 0]) rotate([0, 0, 360 / (30 * 2)]) gear_helix(m = 2, z = 30, x = 0, h = 15, w = 20, w_helix = -30);


In [49]:
with open("../common/helix_gear/gears_script.scad", "w") as f:
    f.write("include <gears.scad>\n")

with open("../common/helix_gear/gears_script.scad", "a") as f:
    for match in matches:
        f.write("\n" + match.strip())

In [50]:
subprocess.Popen(["D:\openSCAD\install\openscad.exe",
                 "../common/helix_gear/gears_script.scad"])

<Popen: returncode: None args: ['D:\\openSCAD\\install\\openscad.exe', '../c...>